<a href="https://colab.research.google.com/github/FernandoGuillen-lab/DataScience/blob/main/Reto_costo_polizas_manipulaci%C3%B3n_de_datos_y_an%C3%A1lisis_estad%C3%ADstico_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [ ]:
reto=pd.read_excel("/content/drive/MyDrive/Colab Notebooks/0.p4sqwlmrs70.ulstcd75coInsurance-Reto.xlsx")

reto.columns

Index(['Edad del contratante', 'Género', 'Indice de masa corporal',
       'Cantidad de hijos', 'Fumador', 'Costo Póliza'],
      dtype='object')

In [ ]:

reto=reto.sort_values(by="Costo Póliza")

m=reto["Costo Póliza"].median()

desv=reto["Costo Póliza"].std()

reto["costo"]=""

for i in range(len(reto)):
  costo= reto['Costo Póliza'].iloc[i]
  if costo <= m - desv:
    reto["costo"].iloc[i] = "bajo"
  elif costo <= m + desv:
    reto["costo"].iloc[i] = "medio"
  else:
     reto["costo"].iloc[i] = "alto"

reto.columns

<ipython-input-5-558abd231654>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reto["costo"].iloc[i] = "medio"
<ipython-input-5-558abd231654>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reto["costo"].iloc[i] = "alto"


Index(['Edad del contratante', 'Género', 'Indice de masa corporal',
       'Cantidad de hijos', 'Fumador', 'Costo Póliza', 'costo'],
      dtype='object')

In [ ]:
reto["conteo"]=1

reto

,Edad del contratante,Género,Indice de masa corporal,Cantidad de hijos,Fumador,Costo Póliza,costo,conteo
804,18,male,23.210,0,no,1121.87390,medio,1
691,18,male,30.140,0,no,1131.50660,medio,1
1060,18,male,33.330,0,no,1135.94070,medio,1
567,18,male,33.660,0,no,1136.39940,medio,1
20,18,male,34.100,0,no,1137.01100,medio,1
...,...,...,...,...,...,...,...,...
973,60,male,32.800,0,yes,52590.82939,alto,1
699,33,female,35.530,0,yes,55135.40209,alto,1
494,31,female,38.095,1,yes,58571.07448,alto,1
1107,45,male,30.360,0,yes,62592.87309,alto,1


In [ ]:
df= reto[["Fumador","conteo","costo"]]

df

,Fumador,conteo,costo
804,no,1,medio
691,no,1,medio
1060,no,1,medio
567,no,1,medio
20,no,1,medio
...,...,...,...
973,yes,1,alto
699,yes,1,alto
494,yes,1,alto
1107,yes,1,alto


In [ ]:
agr= df.groupby(["Fumador", "costo"]).count()

agr

conteo
Fumador costo        
no      alto       45
        medio     862
yes     alto      166
        medio      65

In [ ]:
fumn = agr["conteo"]["no"].to_list()

fums = agr["conteo"]["yes"].to_list()

tabla = pd.DataFrame({
"costo": pd.Series(["alto", "medio"]),
"no": fumn,
"yes": fums
})
tabla = tabla.append({"costo": "Total", "no": tabla['no'].sum(), "yes": tabla['yes'].sum()}, ignore_index=True)
tabla['Total'] = tabla[['no', 'yes']].sum(axis=1)
tabla.set_index("costo", inplace=True)
tabla

<ipython-input-11-04f5a58cdf0a>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabla = tabla.append({"costo": "Total", "no": tabla['no'].sum(), "yes": tabla['yes'].sum()}, ignore_index=True)


,no,yes,Total
costo,,,
alto,45,166,211
medio,862,65,927
Total,907,231,1138


In [ ]:
frecuencias = tabla.copy()
costo = frecuencias.index.to_list()[:-1]
Fumador = frecuencias.columns.to_list()[:-1]
frecuencias = frecuencias[frecuencias.columns].astype(np.float64)
for cos in costo:
  for Fum in Fumador:
    frecuencias[Fum][cos] = frecuencias['Total'][cos] * frecuencias[Fum]['Total'] / frecuencias['Total']['Total']
frecuencias

,no,yes,Total
costo,,,
alto,168.169596,42.830404,211.0
medio,738.830404,188.169596,927.0
Total,907.000000,231.000000,1138.0


In [ ]:
contribuciones = frecuencias.copy()
costo = frecuencias.index.to_list()[:-1]
Fumador = frecuencias.columns.to_list()[:-1]
for cos in costo:
  for Fum in Fumador:
    contribuciones[Fum][cos] = round((tabla[Fum][cos] - frecuencias[Fum][cos]) ** 2 / frecuencias[Fum][cos], 2)
contribuciones.drop('Total', axis=1, inplace=True)
contribuciones.drop('Total', axis=0, inplace=True)
contribuciones.reset_index(inplace=True)
contribuciones = contribuciones.append({"costo": "Total", "Fumador": contribuciones['no'].sum(), "yes": contribuciones['yes'].sum()}, ignore_index=True)
contribuciones['Total'] = contribuciones[['no', 'yes']].sum(axis=1)
contribuciones.set_index('costo', inplace=True)
contribuciones

<ipython-input-13-479e1f3fa3d9>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contribuciones = contribuciones.append({"costo": "Total", "Fumador": contribuciones['no'].sum(), "yes": contribuciones['yes'].sum()}, ignore_index=True)


,no,yes,Fumador,Total
costo,,,,
alto,90.21,354.21,NaN,444.42
medio,20.53,80.62,NaN,101.15
Total,NaN,434.83,110.74,434.83


In [ ]:
subconjunto = contribuciones[Fumador][:-1]
X = round(subconjunto.sum().sum(), 2)
X

545.57

In [ ]:
from scipy.stats import chi2
valor_p = chi2.sf(X, 9)
valor_p

9.915045650764678e-112